In [1]:
from langchain.chains import create_sql_query_chain
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

DEPLOYMENT_NAME="gpt-35-turbo-instruct"
MODEL_NAME = "gpt-35-turbo-instruct"
API_VERSION="2023-12-01-preview"

True

In [2]:
from langchain_community.utilities import SQLDatabase
db = SQLDatabase.from_uri("sqlite:///../data/Chinook.db")
execute_query = QuerySQLDataBaseTool(db=db)
llm = AzureOpenAI(azure_deployment=DEPLOYMENT_NAME, model_name=MODEL_NAME, openai_api_version=API_VERSION)
write_query = create_sql_query_chain(llm, db)

In [4]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

answer = answer_prompt | llm | StrOutputParser()
chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer
)

In [ ]:
chain.invoke({"question": "How many employees are there"})

In [8]:
chain.invoke({"question": "Which invoice has the highest total?"})

' The invoice with InvoiceId 404 has the highest total of $25.86.'

In [5]:
chain.invoke({"question": "List the total sales per country. Which country's customers spent the most?"})

'The country with the most total sales is USA, with a total sales amount of $523.06. '